In [30]:
import os
os.environ['VLLM_USE_V1'] = '0'
from indextts.infer_v2_vllm import IndexTTS2
# from indextts.infer_v2 import IndexTTS2
import asyncio
index_tts = IndexTTS2(cfg_path="checkpoints/config.yaml", model_dir="checkpoints", use_cuda_kernel=False, gpu_memory_utilization=0.25)

>> All random seeds have been set to 42 for reproducibility.
INFO 09-17 09:55:49 [config.py:3131] Downcasting torch.float32 to torch.float16.
INFO 09-17 09:55:49 [config.py:793] This model supports multiple tasks: {'generate', 'reward', 'score', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 09-17 09:55:49 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.0) with config: model='checkpoints/vllm', speculative_config=None, tokenizer='checkpoints/vllm', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1818, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='xgrammar', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-17 09:55:49 [default_loader.py:280] Loading weights took 0.54 seconds
INFO 09-17 09:55:50 [model_runner.py:1202] Model loading took 0.9233 GiB and 0.562023 seconds
INFO 09-17 09:55:51 [worker.py:291] Memory profiling takes 0.39 seconds
INFO 09-17 09:55:51 [worker.py:291] the current vLLM instance can use total_gpu_memory (23.52GiB) x gpu_memory_utilization (0.25) = 5.88GiB
INFO 09-17 09:55:51 [worker.py:291] model weights take 0.92GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.16GiB; the rest of the memory reserved for KV Cache is 4.80GiB.
INFO 09-17 09:55:51 [executor_base.py:112] # cuda blocks: 2619, # CPU blocks: 2184
INFO 09-17 09:55:51 [executor_base.py:117] Maximum concurrency for 1818 tokens per request: 23.05x
INFO 09-17 09:55:53 [model_runner.py:1512] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 09-17 09:56:17 [model_runner.py:1670] Graph capturing finished in 24 secs, took 0.20 GiB
INFO 09-17 09:56:17 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 27.08 seconds
>> GPT weights restored from: checkpoints/gpt.pth
>> semantic_codec weights restored from: ./checkpoints/hf_cache/models--amphion--MaskGCT/snapshots/265c6cef07625665d0c28d2faafb1415562379dc/semantic_codec/model.safetensors
cfm loaded
length_regulator loaded
gpt_layer loaded
>> s2mel weights restored from: checkpoints/s2mel.pth
>> campplus_model weights restored from: ./checkpoints/hf_cache/models--funasr--campplus/snapshots/fb71fe990cbf6031ae6987a2d76fe64f94377b7e/campplus_cn_common.bin
Loading weights from nvidia/bigvgan_v2_22khz_80band_256x


2025-09-17 09:56:29,593 WETEXT INFO found existing fst: /root/RichooAgent/third_party/index-tts/indextts/utils/tagger_cache/zh_tn_tagger.fst
2025-09-17 09:56:29,594 WETEXT INFO                     /root/RichooAgent/third_party/index-tts/indextts/utils/tagger_cache/zh_tn_verbalizer.fst
2025-09-17 09:56:29,594 WETEXT INFO skip building fst for zh_normalizer ...


Removing weight norm...
>> bigvgan weights restored from: nvidia/bigvgan_v2_22khz_80band_256x


2025-09-17 09:56:29,787 WETEXT INFO found existing fst: /venv/main/lib/python3.12/site-packages/tn/en_tn_tagger.fst
2025-09-17 09:56:29,787 WETEXT INFO                     /venv/main/lib/python3.12/site-packages/tn/en_tn_verbalizer.fst
2025-09-17 09:56:29,787 WETEXT INFO skip building fst for en_normalizer ...


>> TextNormalizer loaded
>> bpe model loaded from: checkpoints/bpe.model


In [34]:
import importlib
import types
from pydub import AudioSegment
from indextts import infer_v2_vllm
importlib.reload(infer_v2_vllm)
from indextts.infer_v2_vllm import IndexTTS2
index_tts.infer_stream = types.MethodType(IndexTTS2.infer_stream, index_tts)
index_tts.infer_parallel = types.MethodType(IndexTTS2.infer_parallel, index_tts)

count = 0
concat_segment = None
from io import BytesIO

# async for chunk in index_tts.infer_stream(spk_audio_prompt='data/tina_test.wav', text="""Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.""", output_path="test_parallel2.wav", verbose=False, max_text_tokens_per_segment=120, interval_silence=0):


#     with open(f"test_stream_{count}.wav", "wb") as f:
#         f.write(chunk)
#     if concat_segment is None:
#         concat_segment = AudioSegment.from_file(BytesIO(chunk))
#     else:
#         concat_segment = concat_segment.append(AudioSegment.from_file(BytesIO(chunk)), crossfade=0)

# concat_segment.export("test_stream.mp3", format="mp3")
await index_tts.infer_parallel(spk_audio_prompt='data/test_real.wav', text="""Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.""", output_path="test_parallel.wav", verbose=False, max_text_tokens_per_segment=120, interval_silence=0)


>> Starting parallel inference...
INFO 09-17 10:00:16 [async_llm_engine.py:211] Added request req-12de9c34-817f-4d2f-8ddf-684ffa4fc602.
INFO 09-17 10:00:16 [async_llm_engine.py:211] Added request req-2b84af0a-9d11-458f-a2c7-e301e1ec8bc1.
INFO 09-17 10:00:16 [async_llm_engine.py:211] Added request req-dab655c2-efa2-44ea-8344-4de1caf20a5a.
INFO 09-17 10:00:16 [async_llm_engine.py:211] Added request req-7e9177be-fd61-44b8-9956-7021d4a10ff4.
INFO 09-17 10:00:16 [async_llm_engine.py:211] Added request req-1c27dff2-1d7a-4a85-a0e6-c0bdec578a5c.
INFO 09-17 10:00:16 [async_llm_engine.py:211] Added request req-7849e10d-f9d5-40e9-a2f1-eeac5239e005.


100%|██████████| 20/20 [00:00<00:00, 90.24it/s]


>> GPT generation time (parallel): 1.34 seconds
>> GPT forward pass time: 0.05 seconds
>> S2MEL time: 1.24 seconds
>> BigVGAN time: 0.27 seconds
>> Total inference time: 3.28 seconds
>> Generated audio length: 44.95 seconds
>> Real-Time Factor (RTF): 0.0731
>> WAV file saved to: test_parallel.wav


'test_parallel.wav'

In [ ]:
from pydub import AudioSegment

# 加载两个音频文件
audio1 = AudioSegment.from_mp3("test_stream_1.wav")
audio2 = AudioSegment.from_mp3("test_stream_2.wav")

merged_chunk = audio1.append(audio2, crossfade=0)
merged_chunk.export("merged_chunk.mp3", format="mp3")
# def stream_with_crossfade(audio1, audio2, crossfade_ms=250):
#     # Step 1: 播放 audio1 主体部分（除去最后 crossfade_ms）
#     main_part = audio1[:-crossfade_ms]
#     yield main_part.raw_data  # 或导出为临时文件/分块传输

#     # Step 2: 交叉淡出淡入部分
#     fade_out = audio1[-crossfade_ms:].fade_out(crossfade_ms)
#     fade_in = audio2[:crossfade_ms].fade_in(crossfade_ms)
#     crossfaded = fade_out.overlay(fade_in)  # 混合
#     yield crossfaded.raw_data

#     # Step 3: 播放 audio2 剩余部分
#     remaining = audio2[crossfade_ms:]
#     yield remaining.raw_data
# crossfade_ms = 250
# a1 = audio1[:-crossfade_ms]
# a2 = audio2[crossfade_ms:]
# fade_out = audio1[-crossfade_ms:].fade_out(crossfade_ms)
# fade_in = audio2[:crossfade_ms].fade_in(crossfade_ms)
# crossfaded = fade_out.overlay(fade_in)

# merged_chunk = a1.append(crossfaded, crossfade=0)
# merged_chunk = merged_chunk.append(a2, crossfade=0)
# merged_chunk.export("merged_chunk.mp3", format="mp3")
# with open("merged_chunk.mp3", "wb") as f:
    # f.write(a1 + crossfaded + a2)


In [27]:
import httpx
import requests
from pydub import AudioSegment
from io import BytesIO
import time
payload = {
    "prompt_audio_url": "https://beezstorageblob.blob.core.windows.net/emailblob/cloned_voice_0e3a2d2d-7a24-41d2-b1f5-c7af7f3ab284.wav",
    "text": "Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth, and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKid. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 2-4 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.",
}

concat_segment = None




try:
    with httpx.stream("POST", "http://127.0.0.1:62343/generate-speech-stream/", json=payload, follow_redirects=True) as r:
        # r.raise_for_status() # Ensure the request was successful
        start_time = time.time()
        for chunk in r.iter_bytes():
            end_time = time.time()
            print(f"Time taken to receive chunk: {end_time - start_time:.2f} seconds")
            start_time = end_time
            if concat_segment is None:
                concat_segment = AudioSegment.from_file(BytesIO(chunk))
            else:
                concat_segment = concat_segment.append(AudioSegment.from_file(BytesIO(chunk)), crossfade=0)
    concat_segment.export("test_stream.mp3", format="mp3")
except httpx.HTTPError as e:
    print(f"An HTTP error occurred: {e}")

Time taken to receive chunk: 1.27 seconds
Time taken to receive chunk: 1.13 seconds
Time taken to receive chunk: 1.32 seconds
Time taken to receive chunk: 1.17 seconds
Time taken to receive chunk: 1.25 seconds
Time taken to receive chunk: 1.31 seconds


In [28]:
# transcribe base64
import base64
import requests
# with open("test_stream.mp3", "rb") as f:
with open("test_stream.mp3", "rb") as f:
    data = f.read()
    base64_data = base64.b64encode(data).decode("utf-8")
payload = {
    "audio_data": base64_data,
    "file_extension": "wav",
    "language": "en"
}

response = requests.post("http://127.0.0.1:62343/transcribe_base64/", json=payload)
print(response.json())

{'text': "Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routine at home. Tom could try playing one or two quick games daily, either with family or on a platform like ChessKit. Solving a few chess puzzles each week through an app or workbook is also an excellent way to sharpen his mind. Aiming for around 24 hours of practice a week will build a strong foundation and keep the game enjoyable and engaging for him.", 'segments': [{'start': 0.0, 'end': 29.44, 'text': "Tom could benefit from more consistent practice to really solidify his understanding and skills in chess. This is a common area for growth and with some focused effort, we'll see great progress. To help Tom get more accustomed to chess, we recommend establishing a fun, regular practice routi